In [52]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

In [74]:
cities = ["Adelaide", "Brisbane", "Melbourne", "Perth", "Sydney"]

LCL = dict(zip(cities, [25,20,23,15,22]))

CF = dict(zip(cities, [2000, 1600, 1800, 1200, 1650]))

CH = dict(zip(cities, [1200, 1000, 1100, 700, 1050]))

D = dict(zip(cities, [420, 870, 1250, 930, 1310]))

road_data = {
    'Adelaide': [0, 35, 10, 35, 25],
    'Brisbane': [35, 0, 25, 70, 15],
    'Melbourne': [10, 25, 0, 45, 15],
    'Perth': [35, 70, 45, 0, 55],
    'Sydney': [25, 15, 15, 55, 0]
}

C = pd.DataFrame(road_data, index=['Adelaide', 'Brisbane', 'Melbourne', 'Perth', 'Sydney'])

L = dict(zip(cities, [30, 21, 28, 18, 25]))

p = dict(zip(cities, [450,480, 505, 490, 515]))


O_FC = 440


In [75]:
m = gp.Model()

# VARIABLES

s_f = m.addVars(cities, vtype=GRB.INTEGER, lb=0.0, name='S_F')
s_h = m.addVars(cities, vtype=GRB.INTEGER, lb=0.0, name='S_H')
s_lcl = m.addVars(cities, vtype=GRB.INTEGER, lb=0.0, name='S_LCL')
Q = m.addVars(cities, cities, vtype=GRB.INTEGER, lb=0.0, name='Q')
V = m.addVars(cities, vtype=GRB.INTEGER, lb=0.0, name='V')
b = m.addVars(cities, vtype=GRB.BINARY, name='b')
R = m.addVars(cities, cities, vtype=GRB.INTEGER, lb=0.0, name='R')

m.update()

# CONSTRAINTS

one_city = m.addConstr(gp.quicksum(b[i] for i in cities) == 1, name='one city')

domes_prod = m.addConstrs((V[i] <= 500 * b[i] for i in cities), name='domes production')

domes_capacity = m.addConstrs((gp.quicksum(Q[i, j] for j in cities) <= V[i] for i in cities), name='domes capacity')

demand = m.addConstrs(((200*s_f[i]) + (100*s_h[i]) + s_lcl[i] + gp.quicksum(Q[i, j] for j in cities) + gp.quicksum(R[j, i] for j in cities) - gp.quicksum(R[i, j] for j in cities) >= D[i] for i in cities))

# disallow_road_transport = m.addConstrs(R[i, j] == 0 for i in cities for j in cities)

m.update()

# OBJECTIVE FUNCTION



C_ovis = gp.quicksum(CF[i]*s_f[i] + CH[i]+s_h[i] + LCL[i]*s_lcl[i] for i in cities)

C_domes = gp.quicksum(C.loc[i, j] * Q[i, j] for i in cities for j in cities)

C_transfer = gp.quicksum(C.loc[i, j] * R[i, j] for i in cities for j in cities)

C_ovis_prod = gp.quicksum(440 * ((200*s_f[i]) + (100*s_h[i]) + s_lcl[i]) for i in cities)

C_domes_prod = gp.quicksum(p[i] * V[i] for i in cities)

C_I = gp.quicksum(440 * ((200*s_f[i]) + (100*s_h[i]) + s_lcl[i]) * 0.15 * (L[i] / 365) for i in cities)

m.setObjective(C_ovis + C_domes + C_transfer + C_ovis_prod + C_domes_prod + C_I, GRB.MINIMIZE)
m.update()

In [76]:
# m.setParam('MIPGap', 0.1)

Set parameter MIPGap to value 0.1


In [77]:
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
MIPGap  0.1

Optimize a model with 16 rows, 75 columns and 125 nonzeros
Model fingerprint: 0x0fb99d1b
Variable types: 0 continuous, 75 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e+01, 9e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 2228529.6164
Presolve removed 11 rows and 39 columns
Presolve time: 0.00s
Presolved: 5 rows, 36 columns, 56 nonzeros
Variable types: 0 continuous, 36 integer (0 binary)

Root relaxation: interrupted, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap

In [73]:
m.ObjVal

2149731.8356164386

In [98]:
C_domes.getValue()

0.0

In [102]:
for v in b:
    if b[v].X > 0:
        print(b[v].VarName)

# Build the Domes facility here!

Cost with 0 Domes Locations, and no road transit: `2,132,875.61`

Cost with 0 Domes Locations, and allowing road network transit `2,131,414.06`

Cost with 1 Domes Location, and allowing road network transit: `2,130,763.68`


In [109]:
cost_q1 = 2132875.61
cost_q2 = 2131414.06
cost_q4 = 2130763.68

In [104]:
cost_q1 - cost_q2

1461.5499999998137

In [110]:
cost_q1 - cost_q4

2111.929999999702

In [111]:
cost_q2-cost_q4

650.3799999998882

In [24]:
def nonzeroes(var):  # use this to check results
    for v in var:
        if var[v].X > 0:
            print(var[v].VarName, var[v].X)

In [42]:
nonzeroes(s_h)

S_H[Adelaide] 4.0
S_H[Brisbane] 9.0
S_H[Melbourne] 12.0
S_H[Perth] 9.0
S_H[Sydney] 13.0
